### Лабораторная работа №5

#### <span style="color:blue">Задание.</span> Провести классификацию найденного датасета, методами решающего дерева и случайного леса . В формате Markdown написать пояснения. Объяснить почему были выбраны именно такие гиперпараметры, была ли перекрестная проверка, и т.д.

Сначала мы импортируем необходимые библиотеки и нормализуем данные:

In [8]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import cross_val_score

data = pd.read_csv("adult.data")
column_names = ['age', 'workclass', 'fnlwgt', 'education', 'education-num', 'marital-status', 
                'occupation', 'relationship', 'race', 'sex', 'capital-gain', 'capital-loss', 
                'hours-per-week', 'native-country', 'income']
data.columns = column_names

data = data.replace(' ?', pd.NA)

data = data.dropna()
data = pd.get_dummies(data, columns=['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'native-country'])
data['income'] = data['income'].map({' <=50K': 0, ' >50K': 1})

Разделение данных на обучающий и тестовый наборы

In [9]:
X = data.drop('income', axis=1)
y = data['income']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#### Метод решающего дерева

Создание модели решающего дерева, обучение модели на обучающем наборе и оценка производительности модели на тестовом наборе

In [10]:
decision_tree_model = DecisionTreeClassifier(max_depth=10, random_state=42)

decision_tree_model.fit(X_train, y_train)

y_pred_tree = decision_tree_model.predict(X_test)
accuracy_tree = accuracy_score(y_test, y_pred_tree)
report_tree = classification_report(y_test, y_pred_tree)

print(f'Decision Tree Accuracy: {accuracy_tree}')
print('Decision Tree Classification Report:\n', report_tree)

Decision Tree Accuracy: 0.8448533068125311
Decision Tree Classification Report:
               precision    recall  f1-score   support

           0       0.89      0.91      0.90      4535
           1       0.71      0.64      0.67      1498

    accuracy                           0.84      6033
   macro avg       0.80      0.78      0.79      6033
weighted avg       0.84      0.84      0.84      6033



#### Метод случайного леса

Создание модели случайного леса, обучение модели на обучающем наборе и оценка производительности модели на тестовом наборе

In [11]:
random_forest_model = RandomForestClassifier(n_estimators=100, max_depth=10, random_state=42)

random_forest_model.fit(X_train, y_train)

y_pred_forest = random_forest_model.predict(X_test)
accuracy_forest = accuracy_score(y_test, y_pred_forest)
report_forest = classification_report(y_test, y_pred_forest)

print(f'Random Forest Accuracy: {accuracy_forest}')
print('Random Forest Classification Report:\n', report_forest)

Random Forest Accuracy: 0.8514835073760981
Random Forest Classification Report:
               precision    recall  f1-score   support

           0       0.86      0.95      0.91      4535
           1       0.79      0.55      0.65      1498

    accuracy                           0.85      6033
   macro avg       0.83      0.75      0.78      6033
weighted avg       0.85      0.85      0.84      6033



Для повышения надежности оценки производительности модели, проведем кросс-валидацию(например, с 5 фолдами) для решающего дерева и для случайного леса.

In [12]:
cv_scores_tree = cross_val_score(decision_tree_model, X, y, cv=5)
print('Decision Tree Cross-Validation Scores:', cv_scores_tree)
print(f'Mean Accuracy (Decision Tree): {cv_scores_tree.mean()}')

cv_scores_forest = cross_val_score(random_forest_model, X, y, cv=5)
print('Random Forest Cross-Validation Scores:', cv_scores_forest)
print(f'Mean Accuracy (Random Forest): {cv_scores_forest.mean()}')

Decision Tree Cross-Validation Scores: [0.84485331 0.84930371 0.85278515 0.85576923 0.85593501]
Mean Accuracy (Decision Tree): 0.8517292820521615
Random Forest Cross-Validation Scores: [0.84733963 0.84731432 0.8579244  0.85809019 0.85709549]
Mean Accuracy (Random Forest): 0.8535528070413786


Мы выбрали гиперпараметры (например, max_depth и n_estimators) на основе знаний о данных и провели оценку производительности моделей на тестовом наборе, а также использовали перекрестную проверку для оценки их обобщающей способности.